# Next Word Prediction using GPT 2

## Load a pre-trained Large Language Model (LLM) - GPT-2 model (originally invented by OpenAI), finetune it to a specific text style, optimize it and convert it to TensorFlow Lite

-  [Quelle](https://colab.research.google.com/github/tensorflow/codelabs/blob/main/KerasNLP/io2023_workshop.ipynb#scrollTo=hkj4Tl_gL9by)

### Imports

In [11]:
! pip install keras_nlp
! pip install tensorflow
! pip install tensorflow_datasets 
! pip install tensorflow_text


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [12]:
import numpy as np
import keras_nlp
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_text as tf_text
from tensorflow import keras
from tensorflow.lite.python import interpreter
import time

### Genaerate some Text

In [13]:
gpt2_tokenizer = keras_nlp.models.GPT2Tokenizer.from_preset("gpt2_base_en")
gpt2_preprocessor = keras_nlp.models.GPT2CausalLMPreprocessor.from_preset(
    "gpt2_base_en",
    sequence_length=256,
    add_end_token=True,
)
gpt2_lm = keras_nlp.models.GPT2CausalLM.from_preset("gpt2_base_en", preprocessor=gpt2_preprocessor)

In [17]:
start = time.time()

output = gpt2_lm.generate("My trip to Yosemite was", max_length=100)
print("\nGPT-2 output:")
print(output)  # print(output.numpy().decode("utf-8"))

end = time.time()
print("TOTAL TIME ELAPSED: ", end - start)


/usr/local/lib/python3.11/site-packages/keras/src/ops/nn.py:545: UserWarning: You are using a softmax over axis 3 of a tensor of shape (1, 12, 1, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(



GPT-2 output:

TOTAL TIME ELAPSED:  16.6180739402771


## Next word prediciton 

In [22]:
import keras_nlp
import numpy as np

# Beispiel-Text
input_text = ["Today is a beautiful day"]

# Vorhersage der Logits mit predict()
prediction_logits = gpt2_lm.predict(input_text)

# Ausgabe der Logits-Form
print(f"Logits shape: {prediction_logits.shape}")

# Logits des letzten Tokens extrahieren
last_token_logits = prediction_logits[0, -1, :]  # Letzter Token in der Sequenz

# Top 10 wahrscheinlichste nächsten Token
top_k = 10
top_k_indices = np.argsort(last_token_logits)[-top_k:][::-1]  # Sortiere und wähle Top 10

# Verwende den Tokenizer, um die Token-IDs in Wörter zu dekodieren und bereinige sie
top_k_words = [gpt2_lm.preprocessor.tokenizer.id_to_token(token_id).replace('Ġ', '').replace('Ċ', '') for token_id in top_k_indices]

# Ausgabe der Top 10 nächsten Wörter
print("\nTop 10 predicted next words:")
for i, word in enumerate(top_k_words):
    print(f"{i + 1}: {word}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
Logits shape: (1, 256, 50257)

Top 10 predicted next words:
1: 
2: I
3: The
4: 
5: We
6: Today
7: This
8: A
9: You
10: It


## Top 10 deutsche sprache 

In [26]:
! pip install tf-keras

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 2.9 MB/s eta 0:00:0000:0100:010m

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [27]:
import tensorflow as tf
from transformers import TFAutoModelForCausalLM, AutoTokenizer
import numpy as np

# Lade den deutschen GPT-2 Tokenizer und das Modell
tokenizer = AutoTokenizer.from_pretrained("dbmdz/german-gpt2")
model = TFAutoModelForCausalLM.from_pretrained("dbmdz/german-gpt2")

# Beispiel-Text in Deutsch
input_text = "Heute ist ein schöner Tag"

# Tokenisierung des Eingabetexts
inputs = tokenizer(input_text, return_tensors="tf")

# Vorhersage der Logits mit dem GPT-2 Modell in TensorFlow
outputs = model(**inputs)
logits = outputs.logits

# Extrahiere die Logits des letzten Tokens
last_token_logits = logits[:, -1, :]

# Top 10 wahrscheinlichste nächsten Token
top_k = 10
top_k_indices = tf.math.top_k(last_token_logits, k=top_k).indices.numpy()[0]

# Dekodiere die Token-IDs in Wörter
top_k_words = [tokenizer.decode([token_id]).strip() for token_id in top_k_indices]

# Ausgabe der Top 10 nächsten Wörter
print("\nTop 10 vorhergesagte nächste Wörter:")
for i, word in enumerate(top_k_words):
    print(f"{i + 1}: {word}")



Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2LMHeadModel: ['transformer.h.0.attn.masked_bias', 'transformer.h.1.attn.masked_bias', 'transformer.h.4.attn.masked_bias', 'transformer.h.7.attn.masked_bias', 'transformer.h.3.attn.masked_bias', 'transformer.h.5.attn.masked_bias', 'transformer.h.11.attn.masked_bias', 'transformer.h.8.attn.masked_bias', 'transformer.h.9.attn.masked_bias', 'transformer.h.2.attn.masked_bias', 'transformer.h.10.attn.masked_bias', 'transformer.h.6.attn.masked_bias']
- This IS expected if you are initializing TFGPT2LMHeadModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFGPT2LMHeadModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All t


Top 10 vorhergesagte nächste Wörter:
1: ,
2: .
3: für
4: zum
5: !
6: und
7: in
8: ",
9: ...
10: um
